# Install

# Import

In [4]:
import pinecone
from openai.embeddings_utils import get_embedding
from tqdm import tqdm
import docx
import os
import openai
import json
import pandas as pd 
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings


# The API key for your Azure OpenAI resource.
openai_api_key = os.getenv("OPENAI_API_KEY")

# Mount drive a specify the folder

In [3]:
docs_path = "./data/Function calling.docx"

# Parse Documents

In [4]:
text_chunks = []
doc = docx.Document(docs_path)
for para in doc.paragraphs:
    text_chunks.append(para.text)

In [5]:
# remove all chunks shorter than 10 words and strip the rest
text_chunks = [string.strip().strip('\n') for string in text_chunks if len(string.split()) >= 10]

In [6]:
# Load config values
with open(r'../config.json') as config_file:
    config_details = json.load(config_file)
    
# Setting up the deployment name
deployment_name = config_details['EMBEDDINGS_MODEL']

# This is set to `azure`
openai.api_type = "azure"

# The API key for your Azure OpenAI resource.
openai.api_key = os.getenv("OPENAI_API_KEY")

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
openai.api_base = config_details['OPENAI_API_BASE']

# Currently OPENAI API have the following versions available: 2022-12-01
openai.api_version = config_details['OPENAI_API_VERSION']

# Generate embeddigns

In [32]:
chunks_with_embeddigns = []
i = 1
for chunk in tqdm(text_chunks):
  embedding = get_embedding(chunk, engine=deployment_name)
+(+i)
chunks_with_embeddigns.append({"id":i, , "values": embedding})

100%|██████████| 2/2 [00:11<00:00,  5.64s/it]


In [33]:
    index.upsert(chunks_with_embeddigns)

TypeError: Column `metadata` is expected to be a dictionary, found <class 'str'>

In [22]:
xchunks_with_embeddigns = pd.DataFrame(chunks_with_embeddigns)

chunks_with_embeddigns = xchunks_with_embeddigns

In [25]:
chunks_with_embeddigns.iloc[0]

text         In an API call, you can describe functions to ...
embedding    [-0.02345113828778267, 0.005468207411468029, 0...
Name: 0, dtype: object

# Upload to Pinecone

In [7]:
pinecone.init(
    api_key="f5444e56-58db-42db-afd6-d4bd9b2cb40c",
    environment="asia-southeast1-gcp-free"
)

In [15]:
# create or connect to index
index_name = "function-info-2023"

index = pinecone.Index(index_name)

In [16]:
chunks_with_embeddigns = []
for chunk in tqdm(text_chunks):
  embedding = get_embedding(chunk, engine=deployment_name)
  index.upsert()



chunks_with_embeddigns.append({"id": chunk, "value": embedding})

100%|██████████| 2/2 [00:17<00:00,  8.85s/it]


In [23]:
index.upsert(chunks_with_embeddigns)

TypeError: Column `metadata` is expected to be a dictionary, found <class 'str'>

In [10]:
chunks_with_embeddigns = []

insert_embeddings(index,tqdm(text_chunks) )
  #embedding = get_embedding(chunk, engine=deployment_name)
  #chunks_with_embeddigns.append({"text": chunk, "embedding": embedding})

  0%|          | 0/2 [00:00<?, ?it/s]

TypeError: Pinecone.__init__() got an unexpected keyword argument 'embeddings'

In [41]:
batch_size = 2  # process everything in batches of 64
for i in tqdm(range(0, len(chunks_with_embeddigns), batch_size)):
    data_batch = chunks_with_embeddigns.iloc[i: i+batch_size]
    # set end position of batch
    i_end = min(i+batch_size, len(chunks_with_embeddigns))
    # get batch meta
    text_batch = [item['text'] for item in data_batch]
    # get ids
    ids_batch = [str(n) for n in range(i, i_end)]
    # get embeddings
    embeds = [item['embedding'] for item in data_batch]
    # prep metadata and upsert batch
    meta = [{'text': text_batch} for text_batch in zip(text_batch)] # you can add more fields here
    to_upsert = zip(ids_batch, embeds, meta)
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))

  0%|          | 0/1 [00:00<?, ?it/s]

TypeError: string indices must be integers, not 'str'

# Query Index

In [ ]:
def search_docs(query):
  xq = openai.Embedding.create(input=query, engine="text-embedding-ada-002")['data'][0]['embedding']
  res = index.query([xq], top_k=5, include_metadata=True)
  chosen_text = []
  for match in res['matches']:
    chosen_text = match['metadata']
  return res['matches']

In [ ]:
matches = search_docs("What are some predictions for tiktok?")
for match in matches:
    print(f"{match['score']:.2f}: {match['metadata']}")

# Construct Prompt

In [ ]:
def construct_prompt(query):
  matches = search_docs(query)

  chosen_text = []
  for match in matches:
    chosen_text.append(match['metadata']['text'])

  prompt = """Answer the question as truthfully as possible using the context below, and if the answer is no within the context, say 'I don't know.'"""
  prompt += "\n\n"
  prompt += "Context: " + "\n".join(chosen_text)
  prompt += "\n\n"
  prompt += "Question: " + query
  prompt += "\n"
  prompt += "Answer: "
  return prompt

# Run the model

In [ ]:
def answer_question(query):
  prompt = construct_prompt(query)
  res = openai.Completion.create(
      prompt=prompt,
      model="text-davinci-003",
      max_tokens=500,
      temperature=0.0,
  )

  return res.choices[0].message

In [ ]:
print(answer_question("What will be the top platform in 2023?"))